# Retrieval-Augmented Generation (RAG) system using LangChain and Ollama. The system processes a PDF document,

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Load and split PDF
loader = PyPDFLoader("yolov9_paper.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(data)

# Initialize embeddings with Ollama (free and local)
embedding_model = OllamaEmbeddings(model="mxbai-embed-large")
vectorstore = Chroma.from_documents(docs, embedding=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Initialize LLaMA 3.2 model locally
llm = Ollama(model="llama3.2")

# Define RAG prompt
system_prompt = (
    "You are an AI assistant for question-answering tasks. Use the retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. Keep the answer concise and within three sentences.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# Create RAG pipeline
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Query the model
response = rag_chain.invoke({"input": "What is new in YOLOv9?"})
print(response["answer"])


C:\Users\konde\AppData\Local\Temp\ipykernel_25064\2430642329.py:22: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


According to the text, several new and improved features are present in YOLOv9 compared to its predecessors:

1. **Improved performance**: YOLOv9 achieves state-of-the-art performance on multiple object detection benchmarks, outperforming other popular models such as YOLO MS [7], PP-YOLOE [74], and RT DETR [43].
2. **Mosaic data augmentation**: A new mosaic data augmentation strategy is introduced, which combines the benefits of random flipping, color jittering, and rotation to improve model performance.
3. **Computational efficiency**: YOLOv9 has a more efficient computational structure than its predecessors, with a lower number of parameters and computations required for inference.
4. **Generalized ELAN**: The new ELAN (Efficient Light-weight Anchor Network) module is introduced, which allows for more flexible anchor generation and improves the model's performance on various object detection tasks.
5. **SPP-ELAN**: A new SPP (Single Pyramidal Pooling) module is incorporated into YOLO

# RAG using google api key

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("yolov9_paper.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
#data


In [4]:
len(data)

18

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  96


In [6]:
docs[2]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-03-01T01:40:26+00:00', 'author': '', 'keywords': '', 'moddate': '2024-03-01T01:40:26+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'yolov9_paper.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='GELAN’s architecture confirms that PGI has gained su-\nperior results on lightweight models. We verified the pro-\nposed GELAN and PGI on MS COCO dataset based ob-\nject detection. The results show that GELAN only uses\nconventional convolution operators to achieve better pa-\nrameter utilization than the state-of-the-art methods devel-\noped based on depth-wise convolution. PGI can be used\nfor variety of models from lightweight to large. It can be\nused to obtain complete information, so that train-from-\nscratch models can achieve better res

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

# Set your Google API Key directly
GOOGLE_API_KEY = "your api key"

# Initialize the embedding model with API key
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY))

# Test embedding
# vector = embeddings.embed_query("hello, world!")
print(vector[:5])  # Print first 5 values of the embedding vector


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is new in yolov9?")


In [ ]:
len(retrieved_docs)

In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = "your api key"

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=500,
    google_api_key=GOOGLE_API_KEY
)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is new in YOLOv9?"})
print(response["answer"])